In [0]:
spark.conf.set( "fs.azure.account.key.azuredlssup07.dfs.core.windows.net",
  "access_key_of_account")

In [0]:
dbutils.fs.mount(
  source = "wasbs://azuredls07@azuredlssup07.blob.core.windows.net",
  mount_point = "/mnt/AzureAccount",
  extra_configs = {"fs.azure.account.key.azuredlssup07.blob.core.windows.net":"access_key_of_account"})

True

In [0]:
dbutils.fs.ls("/mnt/AzureAccount/")

[FileInfo(path='dbfs:/mnt/AzureAccount/DatabricksParquet/', name='DatabricksParquet/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/AzureAccount/Pyspark_With_Json/', name='Pyspark_With_Json/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/AzureAccount/YearlyTopTen.csv', name='YearlyTopTen.csv', size=57828, modificationTime=1730564575000),
 FileInfo(path='dbfs:/mnt/AzureAccount/sample1.parquet', name='sample1.parquet', size=1308, modificationTime=1731602962000)]

In [0]:
import json

Json_Data_File = open("/dbfs/mnt/AzureAccount/Pyspark_With_Json/dwsample3.json","rb")
Json_Data = json.loads("/mnt/AzureAccount/Pyspark_With_Json/dwsample3.json")
print(Json_Data)

In [0]:
File_Path = 'abfss://azuredls07@azuredlssup07.dfs.core.windows.net/Pyspark_With_Json/dwsample3.json'
Json_DF_Updated = spark.read.format("json").option("multiline",True).load(File_Path)

In [0]:
display(Json_DF_Updated)

address,age,firstName,gender,lastName,phoneNumbers
"List(San Jone, 394221, CA, 126)",24,Rack,man,Jackon,"List(List(7383627627, home))"


In [0]:
from pyspark.sql.functions import explode_outer

Json_Final = Json_DF_Updated.select("address.*","age","firstName","gender","lastname",explode_outer(Json_DF_Updated.phoneNumbers).alias("PhoneNumber_Updated"))
Json_Final.show()

+--------+----------+-----+-------------+---+---------+------+--------+-------------------+
|    city|postalCode|state|streetAddress|age|firstName|gender|lastname|PhoneNumber_Updated|
+--------+----------+-----+-------------+---+---------+------+--------+-------------------+
|San Jone|    394221|   CA|          126| 24|     Rack|   man|  Jackon| {7383627627, home}|
+--------+----------+-----+-------------+---+---------+------+--------+-------------------+



In [0]:
# display(Json_Final)
Json_Final.select("city","postalCode","state","streetAddress","age","firstName","gender","lastname","PhoneNumber_Updated.*").show()


+--------+----------+-----+-------------+---+---------+------+--------+----------+----+
|    city|postalCode|state|streetAddress|age|firstName|gender|lastname|    number|type|
+--------+----------+-----+-------------+---+---------+------+--------+----------+----+
|San Jone|    394221|   CA|          126| 24|     Rack|   man|  Jackon|7383627627|home|
+--------+----------+-----+-------------+---+---------+------+--------+----------+----+



In [0]:
Json_Sample_DF_2 = spark.read.format("json").option("multiline",True).load("/mnt/AzureAccount/Pyspark_With_Json/sample.json")
display(Json_Sample_DF_2)

CHECK,COL,DATA,IFAM,KTM
"List(1, TWO)",21,"List(List(List(List(k1, v1), List(k2, v2)), 31), List(List(List(k3, v3), List(k4, v4)), 33))",EQR,1548176931466


In [0]:
JSON_TEMP = Json_Sample_DF_2.select("CHECK","COL",explode_outer(Json_Sample_DF_2.DATA).alias("Temp"),"IFAM","KTM")
display(JSON_TEMP)

CHECK,COL,Temp,IFAM,KTM
"List(1, TWO)",21,"List(List(List(k1, v1), List(k2, v2)), 31)",EQR,1548176931466
"List(1, TWO)",21,"List(List(List(k3, v3), List(k4, v4)), 33)",EQR,1548176931466


In [0]:
JSON_TEMP1 = JSON_TEMP.select("CHECK.*","COL","Temp","IFAM","KTM").show()

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-3969828442776096>, line 1
----> 1 JSON_TEMP1 = JSON_TEMP.select("CHECK.*","COL","Temp","IFAM","KTM").show()

AttributeError: 'NoneType' object has no attribute 'select'

In [0]:
from pyspark.sql.types import StructType , StructField ,StringType,IntegerType
from pyspark.sql.functions import from_json, col

PhoneNumbers_schema = StructType([
    StructField("Number" , IntegerType()),
    StructField("home" ,StringType())
])

Json_DF_Updated.withColumn("phoneNumbers_updated",from_json(col("phoneNumbers"),PhoneNumbers_schema))
Json_DF_Updated.drop("phoneNumbers")
Json_DF_Updated.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3969828442776091>, line 9
      2 from pyspark.sql.functions import from_json, col
      4 PhoneNumbers_schema = StructType([
      5     StructField("Number" , IntegerType()),
      6     StructField("home" ,StringType())
      7 ])
----> 9 Json_DF_Updated.withColumn("phoneNumbers_updated",from_json(col("phoneNumbers"),PhoneNumbers_schema))
     10 Json_DF_Updated.drop("phoneNumbers")
     11 Json_DF_Updated.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sq

In [0]:
dbutils.fs.ls('abfss://azuredls07@azuredlssup07.dfs.core.windows.net/Pyspark_With_Json/dwsample3.json')

[FileInfo(path='abfss://azuredls07@azuredlssup07.dfs.core.windows.net/Pyspark_With_Json/dwsample3.json', name='dwsample3.json', size=323, modificationTime=1733595814000)]

In [0]:
import json

# File_Path = 'abfss://azuredls07@azuredlssup07.dfs.core.windows.net/Pyspark_With_Json/dwsample3.json'
# File_Path_Url = "https://azuredlssup07.blob.core.windows.net/azuredls07/Pyspark_With_Json/dwsample3.json"
dbutils.fs.put("/temp/Json_Sample_Data.json",
                "/dbfs/mnt/AzureAccount/Pyspark_With_Json/dwsample3.json", True)
Json_Data = open("/temp/Json_Sample_Data.json", "rb")
Json_data_Dict = json.loads(Json_Data.read())
print(Json_data_Dict)

Wrote 55 bytes.


---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
File <command-355779908317208>, line 7
      3 # File_Path = 'abfss://azuredls07@azuredlssup07.dfs.core.windows.net/Pyspark_With_Json/dwsample3.json'
      4 # File_Path_Url = "https://azuredlssup07.blob.core.windows.net/azuredls07/Pyspark_With_Json/dwsample3.json"
      5 dbutils.fs.put("/temp/Json_Sample_Data.json",
      6                 "/dbfs/mnt/AzureAccount/Pyspark_With_Json/dwsample3.json", True)
----> 7 Json_Data = open("/temp/Json_Sample_Data.json", "rb")
      8 Json_data_Dict = json.loads(Json_Data.read())
      9 print(Json_data_Dict)

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:286, in _modified_open(file, *args, **kwargs)
    279 if file in {0, 1, 2}:
    280     raise ValueError(
    281         f"IPython won't let you open fd={file} by default "
    282         "as it is likely